In [ ]:
import pandas as pd

df_sheet1 = pd.read_excel("Willingness_to_buy.xlsx")
df_sheet = df_sheet1.T
df_sheet2 = pd.read_excel("Pump_prices.xlsx", skiprows=5)
df_sheet3 = pd.read_excel("TIL6010 python.xlsx", sheet_name='Verkoopcijfers autos in nederla')

In [ ]:
import re

def extract_valid_date(date_str):
    match = re.search(r'\d{4}\s\w+', date_str)
    if match:
        return match.group()
    return None


In [ ]:
df_sheet2.rename(columns={"Motor fuel": "Periods"}, inplace=True)
df_sheet2 = df_sheet2[1:]
selected_columns_1 = ["Periods", "Euro 95", "Transport diesel", "LPG"]
df_sheet2 = df_sheet2[selected_columns_1]
df_sheet2['Periods'] = df_sheet2['Periods'].apply(extract_valid_date)
df_sheet2 = df_sheet2.dropna(subset=['Periods'])
df_sheet2['Periods'] = pd.to_datetime(df_sheet2['Periods'], format='%Y %B')
df_sheet2.reset_index(drop=True, inplace=True)
df_sheet2.head()

Periods Euro 95 Transport diesel    LPG
0 2006-01-01   1.357            1.018  0.558
1 2006-02-01   1.330            1.020  0.553
2 2006-03-01   1.356            1.045  0.522
3 2006-04-01   1.431            1.078  0.522
4 2006-05-01   1.433            1.073  0.500

In [ ]:
df_sheet.columns = df_sheet.iloc[0]
df_sheet = df_sheet[2:]
df_sheet.rename(columns={"Topic": "Periods"}, inplace=True)

selected_columns = ["Periods", "Consumer confidence", "Economic climate", "Willingness to buy"]
df_sheet = df_sheet[selected_columns]

df_sheet['Periods'] = df_sheet['Periods'].apply(extract_valid_date)
df_sheet = df_sheet.dropna(subset=['Periods'])
df_sheet['Periods'] = pd.to_datetime(df_sheet['Periods'], format='%Y %B')
df_sheet.reset_index(drop=True, inplace=True)
df_sheet.tail()

Column1    Periods Consumer confidence Economic climate Willingness to buy
445     2023-05-01                 -38              -53                -28
446     2023-06-01                 -39              -54                -29
447     2023-07-01                 -39              -54                -29
448     2023-08-01                 -40              -56                -29
449     2023-09-01                 -39              -57                -28

In [ ]:
month_name_to_number = {
    'januari': 1,
    'februari': 2,
    'maart': 3,
    'april': 4,
    'mei': 5,
    'juni': 6,
    'juli': 7,
    'augustus': 8,
    'september': 9,
    'oktober': 10,
    'november': 11,
    'december': 12
}
df_sheet3['Periods'] = pd.to_datetime(df_sheet3['Jaar'].astype(str) + df_sheet3['Maand'].map(month_name_to_number).astype(str), format='%Y%m')
selected_columns3 = ["Periods", "Verkoop"]
df_sheet3 = df_sheet3[selected_columns3]

df_sheet3.head()

Periods  Verkoop
0 2023-01-01    32845
1 2023-02-01    28128
2 2023-03-01    37452
3 2023-04-01    29669
4 2023-05-01    33135

In [ ]:
result = pd.merge(df_sheet, df_sheet2, on="Periods", how="outer")
result = pd.merge(result, df_sheet3, on="Periods", how="outer")
result.head()

Periods Consumer confidence Economic climate Willingness to buy Euro 95  \
0 1986-04-01                   2                6                  0     NaN   
1 1986-05-01                   8               17                  1     NaN   
2 1986-06-01                  15               33                  3     NaN   
3 1986-07-01                  20               45                  4     NaN   
4 1986-08-01                  21               47                  4     NaN   

  Transport diesel  LPG  Verkoop  
0              NaN  NaN      NaN  
1              NaN  NaN      NaN  
2              NaN  NaN      NaN  
3              NaN  NaN      NaN  
4              NaN  NaN      NaN

In [ ]:
mask = (result['Periods'] >= '2006-01-01') & (result['Periods'] <= '2022-12-31')

# Apply the mask to filter the DataFrame
result = result[mask]

# Reset the index to make it continuous
result.reset_index(drop=True, inplace=True)
result.tail()

Periods Consumer confidence Economic climate Willingness to buy  \
199 2022-08-01                 -54              -74                -40   
200 2022-09-01                 -59              -79                -46   
201 2022-10-01                 -59              -75                -48   
202 2022-11-01                 -56              -72                -46   
203 2022-12-01                 -52              -65                -43   

    Euro 95 Transport diesel    LPG  Verkoop  
199   2.065            1.974  0.849  23638.0  
200   1.967            1.999  0.841  25792.0  
201   1.995            2.076  0.834  28992.0  
202   1.912            1.925  0.824  27810.0  
203   1.759            1.751  0.796  30900.0

## How do car sales in the Netherlands evolve over the years?

To answer the main research question, we need to gain a deeper understanding of the car sales industry. Therefore, we've crafted an informative graph that traces the monthly car sales spanning the years 2006 to 2023. 

An interesting trend emerges when the graph is examined in further detail. Despite the significant differences in peak values between the years 2006 to 2013 and the more recent years, the average car sales over this period have remained relatively consistent. The min-max line for the earlier years shows dramatic maximum and minimum values, which indicates substantial fluctuations. The solid trendline represents the car sales and this trendline remains quite steady during the last years, indicating that, on average, monthly car sales have shown some sort of stability.

This consistency in average car sales suggests that the car sales market, during the recent years, has reached a point of equilibrium where the variations between high and low sales months have quite balanced out over time. This form of stability can be seen as a reflection of a more mature and less turbulent market, which is of great interest to industry analysts and stakeholders.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Load your car sales data from the Excel file
car_sales_data = pd.read_excel('Final_TIL.xlsx')

# Calculate the rolling standard deviation for variance
window_size = 12  # You can adjust the window size as needed
rolling_std = car_sales_data['Verkoop'].rolling(window=window_size).std()

# Plotting the graph with min-max line, trend (average), and variance
plt.figure(figsize=(16, 8))

# Main trend line (average)
plt.plot(car_sales_data['Periods'], car_sales_data['Verkoop'], label='Car Sales', color='blue', linestyle='-')

# Variance shading
plt.fill_between(car_sales_data['Periods'], car_sales_data['Verkoop'] - rolling_std, car_sales_data['Verkoop'] + rolling_std, color='blue', alpha=0.2)

# Annotations for seasons (assuming monthly data)
seasons = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for i, month in enumerate(seasons):
    plt.axvline(x=car_sales_data['Periods'][i], color='red', linestyle='--', linewidth=0.5)
    plt.text(car_sales_data['Periods'][i], car_sales_data['Verkoop'].min(), month, rotation=90, verticalalignment='bottom', color='red')

plt.title('Evolution of Car Sales in the Netherlands Over the Years with Variance and Seasons')
plt.xlabel('Year')
plt.ylabel('Number of Car Sales')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()